In [ ]:
"""
TESTING K MODEL
"""
import cv2
import numpy as np
from keras.models import load_model

# Set the desired image size
image_height = 64
image_width = 64
num_channels = 3

# Load the trained model
model = load_model("trainedK_model.h5")

# Load a foggy image for testing
foggy_image_path = "Fog/Fog/foggy-001.jpg"
foggy_image = cv2.imread(foggy_image_path)

# Resize the foggy image to the desired size
foggy_image = cv2.resize(foggy_image, (image_width, image_height))

# Preprocess the foggy image
foggy_image = foggy_image.astype('float32') / 255.0
foggy_image = np.expand_dims(foggy_image, axis=0)

# Use the model to predict the clear image
clear_image = model.predict(foggy_image)

# Rescale the pixel values to the original range
clear_image = clear_image * 255.0
clear_image = clear_image.astype('uint8')

# Reshape the clear image
clear_image = np.squeeze(clear_image, axis=0)

# Display the clear image
cv2.imshow("Clear Image", clear_image)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [ ]:
# Import necessary libraries
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the folder paths for different types of images
U_folder = "/content/drive/MyDrive/Multi fog/U/"  # Folder containing the U images
L_folder = "/content/drive/MyDrive/Multi fog/L/"  # Folder containing the L images
Llma_folder = "/content/drive/MyDrive/Multi fog/Llma/"  # Folder containing the Llma images
M_folder = "/content/drive/MyDrive/Multi fog/M/"  # Folder containing the M images
K_folder = "/content/drive/MyDrive/Multi fog/K/"  # Folder containing the K images

# Set the desired image size
image_height = 64
image_width = 64

""" K Images Model """
# Load the clear and foggy images
clear_images = []       # List to store clear images
Kfoggy_Images = []      # List to store foggy images
for filename in os.listdir(K_folder):
    if filename.endswith(".png"):
        # Read the foggy image
        K_imgs = cv2.imread(os.path.join(K_folder, filename))
        # Get the corresponding clear image filename
        clear_filename = os.path.splitext(filename)[0] + ".png"
        # Read the clear image
        clear_img = cv2.imread(os.path.join(Llma_folder, clear_filename))
        # Resize the images to the desired dimensions
        K_imgs = cv2.resize(K_imgs, (image_width, image_height))
        clear_img = cv2.resize(clear_img, (image_width, image_height))
        # Append the foggy and clear images to their respective lists
        Kfoggy_Images.append(K_imgs)
        clear_images.append(clear_img)

# Convert the lists of images to numpy arrays
clear_images = np.array(clear_images)
Kfoggy_Images = np.array(Kfoggy_Images)

# Normalize pixel values to range between 0 and 1
clear_images = clear_images.astype('float32') / 255.0
Kfoggy_Images = Kfoggy_Images.astype('float32') / 255.0

# Create the CNN model for K
num_channels = 3     # Number of color channels in the images (RGB)
modelK = Sequential()
modelK.add(Conv2D(256, (3, 3), activation='relu', padding='same', input_shape=(image_width, image_height, num_channels)))
modelK.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
modelK.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
modelK.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
modelK.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
modelK.add(Conv2D(num_channels, (3, 3), activation='relu', padding='same'))

# Compile the model
modelK.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
modelK.fit(Kfoggy_Images, clear_images, epochs=1, batch_size=5)

# Save the trained model
modelK.save('trainedK_model.h5')
